In [1]:
from beir.datasets.data_loader import GenericDataLoader

corpus_path = "/Tmp/lvpoellhuber/datasets/belt/corpus/wikipedia/corpus.jsonl"
query_path = "/Tmp/lvpoellhuber/datasets/msmarco-doc-short/queries.jsonl"
qrels_path = "/Tmp/lvpoellhuber/datasets/msmarco-doc-short/qrels/test.tsv"

corpus, queries, qrels = GenericDataLoader(
    corpus_file=corpus_path, 
    query_file=query_path, 
    qrels_file=qrels_path).load_custom()

100%|██████████| 7250/7250 [00:00<00:00, 16941.62it/s]


In [2]:
corpus

NameError: name 'corpus' is not defined

In [9]:
import sqlite3

def get_article_by_id(db_path, page_id):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    cur.execute("SELECT id, title, text, url FROM articles WHERE id = ?", (page_id,))
    row = cur.fetchone()

    conn.close()
    return row  # row will be (id, title, text, url) or None if not found

def get_article_by_title(db_path, title):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    cur.execute("SELECT title, id, text, url FROM articles WHERE title = ?", (title,))
    row = cur.fetchone()

    conn.close()
    return row  # row or None


# Example usage
db_path = "/Tmp/lvpoellhuber/datasets/belt/corpus/wikipedia/wikipedia.db"
title = "Anarchism"
article = get_article_by_title(db_path, title)

if article:
    print(f"Title: {article[1]}")
    print(f"Text: {article[2]}...")  # Print first 200 chars
else:
    print("Article not found.")


Article not found.


In [16]:
import pandas as pd
from tqdm import tqdm
title_list = set(pd.read_csv("/Tmp/lvpoellhuber/datasets/belt/corpus/wikipedia/doc_titles.csv", header=None, index_col=None)[0].to_list())

missing_articles =  []
for title in tqdm(title_list):
    if not get_article_by_title(db_path, title):
        missing_articles.append(title)

  1%|          | 47884/5238113 [00:03<06:20, 13632.24it/s]


KeyboardInterrupt: 

# NQ

In [1]:
import bz2
import shutil

# Download the dump manually or with requests
# e.g. https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2

# Decompress .bz2 file
with bz2.open("/Tmp/lvpoellhuber/datasets/belt/corpus/wikipedia/enwiki-latest-pages-articles.xml.bz2", "rb") as source, open("/Tmp/lvpoellhuber/datasets/belt/corpus/wikipedia/enwiki-latest-pages-articles.xml", "wb") as dest:
    shutil.copyfileobj(source, dest)

In [18]:
import ir_datasets
import pandas as pd
from tqdm import tqdm
import csv

In [ ]:
dataset = ir_datasets.load("natural-questions")
dataset.docs[0]

NqPassageDoc(doc_id='0-0', text="The trade winds are the prevailing pattern of easterly surface winds found in the tropics , within the lower portion of the Earth 's atmosphere , in the lower section of the troposphere near the Earth 's equator . The trade winds blow predominantly from the northeast in the Northern Hemisphere and from the southeast in the Southern Hemisphere , strengthening during the winter and when the Arctic oscillation is in its warm phase . Trade winds have been used by captains of sailing ships to cross the world 's oceans for centuries , and enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans .", html="<P> The trade winds are the prevailing pattern of easterly surface winds found in the tropics , within the lower portion of the Earth 's atmosphere , in the lower section of the troposphere near the Earth 's equator . The trade winds blow predominantly from the northeast in the Northern 

In [72]:
with open("/Tmp/lvpoellhuber/datasets/temp/nq_ids.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "url"])
    for i, doc in enumerate(tqdm(dataset.docs_iter(), total=dataset.docs_count())):
        if  doc.doc_id.split("-")[1]=="0":
            writer.writerow([doc.document_title, doc.document_url])
            


100%|██████████| 28390850/28390850 [01:04<00:00, 442283.59it/s]


# HotPotQA

In [3]:
import ir_datasets
from tqdm import tqdm
dataset = ir_datasets.load("beir/hotpotqa")


In [23]:
for doc in dataset.docs_iter():
    break

In [4]:
all_doc_ids = set()
for doc in tqdm(dataset.docs_iter()):
    all_doc_ids.add(doc.doc_id)

5233329it [00:10, 511164.65it/s]


In [1]:
import sqlite3

def get_article_by_id(db_path, page_id):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    cur.execute("SELECT id, title, text, url FROM articles WHERE id = ?", (page_id,))
    row = cur.fetchone()

    conn.close()
    return row  # row will be (id, title, text, url) or None if not found

def query_articles_by_ids(db_path, id_list):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    results = []
    batch_size = 500  # safe batch size under SQLite limits

    for i in tqdm(range(0, len(id_list), batch_size)):
        batch = id_list[i:i + batch_size]
        placeholders = ",".join("?" for _ in batch)
        query = f"SELECT id, title, text, url FROM articles WHERE id IN ({placeholders})"
        cur.execute(query, batch)
        results.extend(cur.fetchall())

    conn.close()
    return results

# Example usage
db_path = "/Tmp/lvpoellhuber/datasets/belt/corpus/wikipedia/wikipedia.db"

# query_articles_by_ids(db_path, list(all_doc_ids))

In [5]:
results = get_article_by_id(db_path, 12) 
results

In [ ]:
missing_articles

In [77]:
with open("/Tmp/lvpoellhuber/datasets/temp/hpqa_ids.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "url"])
    for i, doc in enumerate(tqdm(dataset.docs_iter(), total=dataset.docs_count())):
        writer.writerow([doc.title, doc.url])
            


100%|██████████| 5233329/5233329 [00:19<00:00, 263378.99it/s]


In [80]:
hpqa_doc_df = pd.read_csv("/Tmp/lvpoellhuber/datasets/temp/hpqa_ids.csv", header=0, index_col=None)
hpqa_doc_df

,title,url
0,Anarchism,https://en.wikipedia.org/wiki?curid=12
1,Autism,https://en.wikipedia.org/wiki?curid=25
2,Albedo,https://en.wikipedia.org/wiki?curid=39
3,A,https://en.wikipedia.org/wiki?curid=290
4,Alabama,https://en.wikipedia.org/wiki?curid=303
...,...,...
5233324,2017 ATM FA season,https://en.wikipedia.org/wiki?curid=55408463
5233325,Philip MacDonald (athlete),https://en.wikipedia.org/wiki?curid=55408472
5233326,Hannah T. King,https://en.wikipedia.org/wiki?curid=55408493
5233327,Pierre Arnaudin,https://en.wikipedia.org/wiki?curid=55408496


In [82]:
nq_doc_df = pd.read_csv("/Tmp/lvpoellhuber/datasets/temp/nq_ids.csv", header=0, index_col=None)
nq_doc_df

,title,url
0,Trade winds,https://en.wikipedia.org//w/index.php?title=Tr...
1,High School Musical 2,https://en.wikipedia.org//w/index.php?title=Hi...
2,List of Nobel laureates in Physics,https://en.wikipedia.org//w/index.php?title=Li...
3,Alice in Wonderland (1951 film),https://en.wikipedia.org//w/index.php?title=Al...
4,Deadpool 2,https://en.wikipedia.org//w/index.php?title=De...
...,...,...
231690,The Conjuring Universe,https://en.wikipedia.org//w/index.php?title=Th...
231691,Brandon Jenkins (musician),https://en.wikipedia.org//w/index.php?title=Br...
231692,A Kiss to Build a Dream On,https://en.wikipedia.org//w/index.php?title=A_...
231693,Fuse (electrical),https://en.wikipedia.org//w/index.php?title=Fu...


In [97]:
nq_set = set(nq_doc_df["title"])
hpqa_set = set(hpqa_doc_df["title"])

overlap = nq_set.intersection(hpqa_set)
len(overlap)

104927

In [101]:
all_titles = pd.concat([nq_doc_df["title"], hpqa_doc_df["title"]]).unique()
len(all_titles)

5238117

In [102]:
5238117-len(hpqa_doc_df)

4788

In [94]:
def wiki_title_to_url(title):
    return "_".join(title.split(" "))

# WikIR

In [ ]:
import os
import requests
from tqdm import tqdm

LOCAL_PATH = "/Tmp/lvpoellhuber/datasets/wikir/"
os.makedirs(LOCAL_PATH, exist_ok=True)

file_url = "https://zenodo.org/records/3707238/files/enwikIRS.zip?download=1"

local_path = os.path.join(LOCAL_PATH, "DORIS-enwikIRS.zip")

with requests.get(file_url, stream=True) as r:
    r.raise_for_status()
    with open(local_path, 'wb') as f:
        for chunk in tqdm(r.iter_content(chunk_size=8192)):
            f.write(chunk) 

In [128]:
import zipfile
import pandas as pd
import io

# Open the ZIP file in read mode ('r') using a context manager
with zipfile.ZipFile('/Tmp/lvpoellhuber/datasets/wikir/DORIS-enwikIRS.zip', 'r') as zf:
    # List all files and directories within the archive
    print("Files in the archive:")
    for file_info in zf.infolist():
        print(f"  - {file_info.filename}")

    # You can also get a list of just the filenames
    print("\nFilenames in the archive:")
    print(zf.namelist())

    zf.extract('enwikIRS/documents.csv', "/Tmp/lvpoellhuber/datasets/wikir")
    # wikir_df = pd.Series()
    # with zf.open('enwikIRS/documents.csv') as f:
    #     text_file = io.TextIOWrapper(f, encoding='utf-8')
    #     for i, line in enumerate(text_file.readlines()):
    #         wikir_df[i] = line.count(" ")+1

    #         if i == 250: 
    #             break


Files in the archive:
  - enwikIRS/
  - enwikIRS/validation/
  - enwikIRS/validation/queries.csv
  - enwikIRS/validation/qrels
  - enwikIRS/validation/BM25.res
  - enwikIRS/validation/BM25.metrics.json
  - enwikIRS/validation/BM25.qrels.csv
  - enwikIRS/test/
  - enwikIRS/test/queries.csv
  - enwikIRS/test/qrels
  - enwikIRS/test/BM25.res
  - enwikIRS/test/BM25.qrels.csv
  - enwikIRS/test/BM25.metrics.json
  - enwikIRS/documents.csv
  - enwikIRS/training/
  - enwikIRS/training/BM25.metrics.json
  - enwikIRS/training/queries.csv
  - enwikIRS/training/qrels
  - enwikIRS/training/BM25.res
  - enwikIRS/training/BM25.qrels.csv

Filenames in the archive:
['enwikIRS/', 'enwikIRS/validation/', 'enwikIRS/validation/queries.csv', 'enwikIRS/validation/qrels', 'enwikIRS/validation/BM25.res', 'enwikIRS/validation/BM25.metrics.json', 'enwikIRS/validation/BM25.qrels.csv', 'enwikIRS/test/', 'enwikIRS/test/queries.csv', 'enwikIRS/test/qrels', 'enwikIRS/test/BM25.res', 'enwikIRS/test/BM25.qrels.csv', 'e

In [156]:
import json

with open("/Tmp/lvpoellhuber/datasets/wikir/documents.csv") as f:
    print("Counting rows...")
    rowcount = sum(1 for line in f)

with open("/Tmp/lvpoellhuber/datasets/wikir/documents.csv") as f:
    with open("/Tmp/lvpoellhuber/datasets/wikir/documents.json", "w") as file:
        print("Reading lines...")
        for i, line in enumerate(tqdm(f.readlines(), total=rowcount)):
            if i==0:
                print("Writing to JSON.")
            else:
                line = line.split(",")
                json.dump(
                    {
                        "id": line[0], 
                        "text": line[1].strip()
                    }, 
                    file,
                    indent=4
                )


Counting rows...
Reading lines...


  0%|          | 213/2456638 [00:00<26:08, 1566.28it/s]

Writing to JSON.


100%|██████████| 2456638/2456638 [00:41<00:00, 59905.27it/s]


In [147]:
line.split(",")[1].strip()

"These institutions are often described as stateless societies although several authors have defined them more specifically as distinct institutions based on non-hierarchical or free associations Anarchism's central disagreement with other ideologies is that it holds the state to be undesirable unnecessary and harmful Anarchism is usually placed on the far-left of the political spectrum and much of its economics and legal philosophy reflect anti-authoritarian interpretations of communism collectivism syndicalism mutualism or participatory economics As anarchism does not offer a fixed body of doctrine from a single particular worldview many anarchist types and traditions exist and varieties of anarchy diverge widely Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of social and individualist anarchism or similar dual classifications The etymologic

In [2]:
import json

with open("/Tmp/lvpoellhuber/datasets/wikir/documents.json", "r") as f:
    documents = json.load(f)

JSONDecodeError: Extra data: line 4 column 2 (char 41493)

In [ ]:
documents

# SciDocs

### IR Datasets

In [1]:
import ir_datasets
dataset = ir_datasets.load("beir/scidocs")
for doc in dataset.docs_iter():
    break
doc

BeirSciDoc(doc_id='632589828c8b9fca2c3a59e97451fde8fa7d188d', text='An evolutionary recurrent network which automates the design of recurrent neural/fuzzy networks using a new evolutionary learning algorithm is proposed in this paper. This new evolutionary learning algorithm is based on a hybrid of genetic algorithm (GA) and particle swarm optimization (PSO), and is thus called HGAPSO. In HGAPSO, individuals in a new generation are created, not only by crossover and mutation operation as in GA, but also by PSO. The concept of elite strategy is adopted in HGAPSO, where the upper-half of the best-performing individuals in a population are regarded as elites. However, instead of being reproduced directly to the next generation, these elites are first enhanced. The group constituted by the elites is regarded as a swarm, and each elite corresponds to a particle within it. In this regard, the elites are enhanced by PSO, an operation which mimics the maturing phenomenon in nature. These enhan

In [2]:
for query in dataset.queries_iter():
    break
query

BeirSciQuery(query_id='78495383450e02c5fe817e408726134b3084905d', text='A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect', authors=['50306438', '15303316', '1976596'], year=2014, cited_by=['38e78343cfd5c013decf49e8cf008ddf6458200f'], references=['632589828c8b9fca2c3a59e97451fde8fa7d188d', '4cf296b9d4ef79b838dc565e6e84ab9b089613de', '86e87db2dab958f1bd5877dc7d5b8105d6e31e46', '4b031fa8bf63e17e2100cf31ba6e11d8f80ff2a8', 'a718c6ca7a1db49bb2328d43f775783e8ec6f985', 'cf51cfb5b221500b882efee60b794bc11635267e', '6329874126a4e753f98c40eaa74b666d0f14eaba', 'a27b6025d147febb54761345eafdd73954467aca'])

### S2ORC

In [ ]:
import json
import os
import re
import requests
import wget
from tqdm import tqdm

import dotenv
dotenv.load_dotenv()

# modify these
API_KEY = os.getenv("S2_API_KEY")
DATASET_NAME = "s2orc"
LOCAL_PATH = "/Tmp/lvpoellhuber/datasets/s2orc/"
os.makedirs(LOCAL_PATH, exist_ok=True)

# get latest release's ID
response = requests.get("https://api.semanticscholar.org/datasets/v1/release/latest").json()
RELEASE_ID = response["release_id"]
print(f"Latest release ID: {RELEASE_ID}")

# get the download links for the s2orc dataset; needs to pass API key through `x-api-key` header
# download via wget. this can take a while...
response = requests.get(f"https://api.semanticscholar.org/datasets/v1/release/{RELEASE_ID}/dataset/{DATASET_NAME}/", headers={"x-api-key": API_KEY}).json()
for url in tqdm(response["files"]):
    match = re.match(r"https://ai2-s2ag.s3.amazonaws.com/staging/(.*)/s2orc/(.*).gz(.*)", url)
    assert match.group(1) == RELEASE_ID
    SHARD_ID = match.group(2)
    wget.download(url, out=os.path.join(LOCAL_PATH, f"{SHARD_ID}.gz"))
print("Downloaded all shards.")

In [6]:
import gzip
import json

with gzip.open('/Tmp/lvpoellhuber/datasets/s2orc/20250725_113419_00148_vtvw6_00b77987-7803-423e-a3f0-6616b2a3fd8e.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        break


In [12]:
article = json.loads(line)
article["content"]["text"].count(" ")+1

11559

In [14]:
article["corpusid"]

253983457

# BioASQ

In [ ]:
import json
import os
import res
import requests
import wget
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import dotenv
dotenv.load_dotenv()

LOCAL_PATH = "/Tmp/lvpoellhuber/datasets/bioasq/"
os.makedirs(LOCAL_PATH, exist_ok=True)

pubmed_directories = [
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_noncomm/xml/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_other/xml/"
]


for directory in pubmed_directories:
    response = requests.get(directory)
    soup = BeautifulSoup(response.text, "html.parser")

    # Step 2: Extract all .tar.gz links
    links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.tar.gz')]

    # Step 3: Download each file
    for link in tqdm(links, desc="Downloading files"):
        file_url = urljoin(directory, link)
        local_path = os.path.join(LOCAL_PATH, link)

        if os.path.exists(local_path):
            print(f"Already downloaded: {link}")
            continue

        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk) 

KeyboardInterrupt: 

In [ ]:
import os
import requests
from bs4 import BeautifulSocup
from urllib.parse import urljoin
from tqdm import tqdm

# URL of the page listing the PMC dump files
BASE_URL = "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/"

# Folder to save downloaded files
OUTPUT_DIR = "pmc_oa_bulk"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Step 1: Get and parse the webpage
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text, "html.parser")

# Step 2: Extract all .tar.gz links
links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.tar.gz')]

# Step 3: Download each file
for link in tqdm(links, desc="Downloading files"):
    file_url = urljoin(BASE_URL, link)
    local_path = os.path.join(OUTPUT_DIR, link)

    if os.path.exists(local_path):
        print(f"Already downloaded: {link}")
        continue

    with requests.get(file_url, stream=True) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

In [ ]:
response = requests.get("https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/")
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"\n  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\n<head>\n<title>Object not found!</title>\n<link rev="made" href="mailto:%5bno%20address%20given%5d" />\n<style type="text/css"><!--/*--><![CDATA[/*><!--*/ \n    body { color: #000000; background-color: #FFFFFF; }\n    a:link { color: #0000CC; }\n    p, address {margin-left: 3em;}\n    span {font-size: smaller;}\n/*]]>*/--></style>\n</head>\n\n<body>\n<h1>Object not found!</h1>\n<p>\n\n\n    The requested URL was not found on this server.\n\n  \n\n    If you entered the URL manually please check your\n    spelling and try again.\n\n  \n\n</p>\n<p>\nIf you think this is a server error, please contact\nthe <a href="mailto:%5bno%20address%20given%5d">webmaster</a>.\n\n</p>\n\n<h2>Error 404</h2>\n<address>\n  <a href="/">ftp.ncbi.nlm.nih.gov</a><br />\n  <span>Apache</s

In [16]:
import pubmed_parser as pp

In [ ]:
dict_out = pp.parse_pubmed_xml(path)

# Gov2
Need TREC...

In [47]:
import ir_datasets
dataset = ir_datasets.load("gov2/trec-mq-2007")
for doc in dataset.docs_iter():
    break

[INFO] GOV2 is available by hard drive from UoG here: <http://ir.dcs.gla.ac.uk/test_collections/access_to_data.html>
More details about the procedure can be found here: <https://ir-datasets.com/gov2.html#DataAccess>.
Link the GOV2 source files here: /u/poellhul/.ir_datasets/gov2/corpus
Should contain GOV2_data


FileNotFoundError: /u/poellhul/.ir_datasets/gov2/corpus

In [45]:
doc

NameError: name 'doc' is not defined

# ClueWeb
Also need TREC...

In [48]:
import ir_datasets
dataset = ir_datasets.load("clueweb09/catb/trec-web-2009/diversity")
for doc in dataset.docs_iter():
    break

[INFO] ClueWeb09 is available by hard drives from CMU here: <https://lemurproject.org/clueweb09/>
More details about the procedure can be found here: <https://ir-datasets.com/ClueWeb09.html#DataAccess>.
Link the ClueWeb09 source files here: /u/poellhul/.ir_datasets/clueweb09/corpus
Should contain directories like ClueWeb09_English_1


FileNotFoundError: /u/poellhul/.ir_datasets/clueweb09/corpus

# DORIS-MAE

In [ ]:
import os
import requests

LOCAL_PATH = "/Tmp/lvpoellhuber/datasets/doris-mae/"
os.makedirs(LOCAL_PATH, exist_ok=True)

file_url = "https://zenodo.org/records/8299749/files/DORIS-MAE_dataset_v1.json?download=1"

local_path = os.path.join(LOCAL_PATH, "DORIS-MAE_dataset_v1.json")

with requests.get(file_url, stream=True) as r:
    r.raise_for_status()
    with open(local_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk) 

In [ ]:
import json
import os
import requests

LOCAL_PATH = "/Tmp/lvpoellhuber/datasets/doris-mae/"
os.makedirs(LOCAL_PATH, exist_ok=True)
local_path = os.path.join(LOCAL_PATH, "DORIS-MAE_dataset_v1.json")


In [7]:
with open(local_path, 'r') as file:
    dataset = json.load(file)

In [12]:
corpus_sample = dataset["Corpus"][:10]

In [29]:
pdf_dir = os.path.join(LOCAL_PATH, "pdfs")
os.makedirs(pdf_dir, exist_ok=True)
download_url = "https://arxiv.org/pdf/"

for sample in corpus_sample:
    url = sample["url"]
    doi = url.split("/")[-1]

    pdf_url = download_url+doi
    response = requests.get(pdf_url)
    
    pdf_path = os.path.join(pdf_dir, doi)+".pdf"
    with open(pdf_path, "wb") as f:
        f.write(response.content)


In [32]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

dir = "/Tmp/lvpoellhuber/datasets/doris-mae/pdfs"

for pdf in os.listdir(dir):
    text = extract_text_from_pdf(os.path.join(dir, pdf))

In [33]:
text

'Statistical Program Slicing: a Hybrid Slicing Technique\nfor Analyzing Deployed Software\nBogdan Alexandru Stoica*\nUniversity of Chicago\nSwarup K. Sahoo\nDeepFence Inc.\nJames R. Larus\nEPFL\nVikram S. Adve\nUniversity of Illinois at Urbana-Champaign\nABSTRACT\nDynamic program slicing can significantly reduce the code\ndevelopers need to inspect by narrowing it down to only a\nsubset of relevant program statements. However, despite an\nextensive body of research showing its usefulness, dynamic\nslicing is still short from production-level use due to the high\ncost of runtime instrumentation.\nAs an alternative, we propose statistical program slicing, a\nnovel hybrid dynamic-static slicing technique that explores the\ntrade-off between accuracy and runtime cost. Our approach\nrelies on modern hardware support for control flow monitoring\nand a novel, cooperative heap memory tracing mechanism com-\nbined with static program analysis for data flow tracking. We\nevaluate statistical sli

# CORD-19

In [ ]:
import os
import requests
from tqdm import tqdm

LOCAL_PATH = "/Tmp/lvpoellhuber/datasets/cord19/"
os.makedirs(LOCAL_PATH, exist_ok=True)

# There are more URLs
file_url = "https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-04-10.tar.gz"

local_path = os.path.join(LOCAL_PATH, "cord-19_2020-04-10.tar.gz")

with requests.get(file_url, stream=True) as r:
    r.raise_for_status()
    with open(local_path, 'wb') as f:
        for chunk in tqdm(r.iter_content(chunk_size=8192)):
            f.write(chunk) 

In [ ]:
import tarfile
import json
import pandas as pd
from tqdm import tqdm

tar_path = "/Tmp/lvpoellhuber/datasets/cord19/cord-19_2020-04-10.tar.gz"

cord19_df = pd.Series()
with tarfile.open(tar_path, "r:bz2") as tar:
    inner_member = tar.getmember("2020-04-10/biorxiv_medrxiv.tar.gz")
    with tar.extractfile(inner_member) as inner_file:
        inner_bytes = io.BytesIO(inner_file.read())

        # Step 2: Open the inner tar.gz file
        with tarfile.open(fileobj=inner_bytes, mode='r:gz') as inner_tar:
            # Step 3: Access the target file
            try:
                target_member = inner_tar.getmember(target_path)
                with inner_tar.extractfile(target_member) as f:
                    content = f.read().decode('utf-8')  # or f.readlines(), etc.
                    print(content[:500])  # Print the first 500 characters
            except KeyError:
                print(f"{target_path} not found in {inner_name}")

Found inner archive: 2020-04-10/biorxiv_medrxiv.tar.gz
?rw-r--r-- lucyw/users      54702 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/f056da9c64fbf00a4645ae326e8a4339d015d155.json 
?rw-r--r-- lucyw/users     202709 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/daf32e013d325a6feb80e83d15aabc64a48fae33.json 
?rw-r--r-- lucyw/users     126724 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/f33c6d94b0efaa198f8f3f20e644625fa3fe10d2.json 
?rw-r--r-- lucyw/users     110737 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/4da8a87e614373d56070ed272487451266dce919.json 
?rw-r--r-- lucyw/users     131344 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/eccef80cfbe078235df22398f195d5db462d8000.json 
?rw-r--r-- lucyw/users     114336 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/c41fdb2efd6d61384a92a84cbba3f8233629a41b.json 
?rw-r--r-- lucyw/users      90395 2020-04-10 16:06:55 biorxiv_medrxiv/pdf_json/1dd898b5ca1ae70ec0e3cad89fc87a165002a99e.json 
?rw-r--r-- lucyw/users      62917 2020-04-10 16:06:55 biorxiv_m

KeyboardInterrupt: 

# NFCorpus

In [ ]:
import ir_datasets
dataset = ir_datasets.load("nfcorpus")
for doc in dataset.docs_iter():
    break
doc

[INFO] If you have a local copy of https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz, you can symlink it here to avoid downloading it again: /u/poellhul/.ir_datasets/downloads/49c061fbadc52ba4d35d0e42e2d742fd
[INFO] [starting] https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz
[INFO] [finished] https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz: [00:05] [31.0MB] [5.65MB/s]


NfCorpusDoc(doc_id='MED-1', url='http://www.ncbi.nlm.nih.gov/pubmed/23092936', title='Birth Weight, Head Circumference, and Prenatal Exposure to Acrylamide from Maternal Diet: The European Prospective Mother–Child Study (NewGeneris)', abstract='Abstract Background: Acrylamide is a common dietary exposure that crosses the human placenta. It is classified as a probable human carcinogen, and developmental toxicity has been observed in rodents. Objectives: We examined the associations between prenatal exposure to acrylamide and birth outcomes in a prospective European mother–child study. Methods: Hemoglobin (Hb) adducts of acrylamide and its metabolite glycidamide were measured in cord blood (reflecting cumulated exposure in the last months of pregnancy) from 1,101 singleton pregnant women recruited in Denmark, England, Greece, Norway, and Spain during 2006–2010. Maternal diet was estimated through food-frequency questionnaires. Results: Both acrylamide and glycidamide Hb adducts were asso

In [31]:
doc.url

'http://www.ncbi.nlm.nih.gov/pubmed/23092936'